In [1]:
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import naive_bayes

In [2]:
from nltk import word_tokenize
from nltk.classify import NaiveBayesClassifier
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import extract_unigram_feats, mark_negation

C:\Users\Tejal\Anaconda2\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
from unidecode import unidecode

In [4]:
from nltk.corpus import stopwords

In [5]:
s = set(stopwords.words('english'))
vec = TfidfVectorizer(use_idf=True, lowercase=True,strip_accents='ascii',stop_words=s )

In [16]:
data = pd.read_csv(r'C:\Users\Tejal\Documents\Tejal\College\SSCI580\Project\Sentiment analysis\Train1\Train1.csv',names = ["Sentiment", "c1", "c2", "c3","c4","tweet"],encoding='latin-1')

In [242]:
data.axes()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [244]:
data.groupby('Sentiment').count()

,c1,c2,c3,c4,tweet
Sentiment,,,,,
0,800000,800000,800000,800000,800000
4,800000,800000,800000,800000,800000


In [17]:
train_data = data[["Sentiment","tweet"]].copy()

In [87]:
train_data.head()

,Sentiment,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [13]:
train_data.shape

(1600000, 2)

In [8]:
import re

In [18]:
train_data['tweet']=train_data['tweet'].apply(lambda x: re.sub(r"http\S+", "", x))
train_data['tweet']=train_data['tweet'].apply(lambda x: re.sub(r"^@\S+", "", x))

In [24]:
train_data.head()

,Sentiment,tweet
0,0,"- Awww, that's a bummer. You shoulda got Da..."
1,0,is upset that he can't update his Facebook by ...
2,0,I dived many times for the ball. Managed to s...
3,0,my whole body feels itchy and like its on fire
4,0,"no, it's not behaving at all. i'm mad. why am..."


In [31]:
mark_negation(train_data.iloc[1,1].split()) 

['is',
 'upset',
 'that',
 'he',
 "can't",
 'update_NEG',
 'his_NEG',
 'Facebook_NEG',
 'by_NEG',
 'texting_NEG',
 'it..._NEG',
 'and_NEG',
 'might_NEG',
 'cry_NEG',
 'as_NEG',
 'a_NEG',
 'result_NEG',
 'School_NEG',
 'today_NEG',
 'also._NEG',
 'Blah!_NEG']

In [20]:
import sklearn.utils
train_data = sklearn.utils.shuffle(train_data)
train_data = train_data.reset_index(drop=True)

In [21]:
train_data1 = train_data.iloc[0:40000].copy()

In [22]:
train_data1.groupby('Sentiment').count()

,tweet
Sentiment,
0,20090
4,19910


In [24]:
def clean_text(text):
    text = text.replace("<br />", " ")
    text = text.decode("utf-8")
 
    return text
clean_text(train_data.iloc[0,1])

u' haha likewise.  I always forget how different our time zones are. Perhaps I shall hunt you down on facebook now. '

In [33]:
analyzer = SentimentAnalyzer()
voc = analyzer.all_words([mark_negation(word_tokenize(unidecode(clean_text(instance.encode('ascii', 'ignore').decode('ascii').encode("utf-8")))))
                                 for instance in train_data1.iloc[:,1]])
print "Computing Unigran Features ..."
unigram_features = analyzer.unigram_word_feats(voc, min_freq=10)


Computing Unigran Features ...


In [30]:
train_data1.iloc[86,1].encode('ascii', 'ignore').decode('ascii').encode('utf-8')

' mon pneu est crev '

In [34]:
len(unigram_features)

3850

In [35]:
analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_features)

In [36]:
train_X = analyzer.apply_features([mark_negation(word_tokenize(unidecode(clean_text(instance.encode('ascii', 'ignore').decode('ascii').encode("utf-8"))))) 
                                    for instance in train_data1.iloc[:,1]], labeled=False)

In [37]:
trainer = NaiveBayesClassifier.train
classifier = analyzer.train(trainer, zip(train_X, train_data1.iloc[:,0]))

Training classifier


In [38]:
train_data1.iloc[0:10,0]

0    4
1    0
2    0
3    0
4    0
5    4
6    4
7    0
8    4
9    0
Name: Sentiment, dtype: int64

In [40]:
pos = 0
neg =0
for ins in train_data.iloc[200000:200015,1]:
    v = analyzer.classify(ins)
    if v ==4:
        pos=pos+1
    else:
        neg=neg+1
        

In [71]:
p = pd.DataFrame(train_data.iloc[200001:200200])
p['new']=""


In [72]:
p.shape

(199, 3)

In [73]:
for i in range(0,199):
    s = analyzer.classify(p.iloc[i,1])
    p.iloc[i,2]=s   
    

In [76]:
p.groupby('Sentiment').count()

,tweet,new
Sentiment,,
0,91,91
4,108,108


In [77]:
20/91.0

0.21978021978021978

In [75]:
p.groupby('new').count()

,Sentiment,tweet
new,,
0,20,20
4,179,179


In [39]:
train_data.iloc[200000:200015].groupby('Sentiment').count()

,tweet
Sentiment,
0,6
4,9


In [40]:
test = train_data.iloc[200000:200015].copy()

In [ ]:
test_X = analyzer.apply_features([mark_negation(word_tokenize(unidecode(clean_text(instance.decode('latin-1').encode("utf-8"))))) 
                                   for instance in test.iloc[:,1]], labeled=False)

In [251]:
len(test_X)

200

In [256]:
analyzer.evaluate(zip(test_X, train_data.iloc[200000:200015,0]))

Evaluating NaiveBayesClassifier results...


{'Accuracy': 0}

In [124]:
acc_data = pd.read_csv(r'C:\Users\Tejal\Documents\Tejal\College\SSCI580\Project\all_usa_tweets.csv',encoding='latin-1')

In [125]:
acc_data['Tweet content']=acc_data['Tweet content'].apply(lambda x: re.sub(r"http\S+", "", x))
acc_data['Tweet content']=acc_data['Tweet content'].apply(lambda x: re.sub(r"^@\S+", "", x))

In [126]:
acc_data["neg"]=""
acc_data["neu"]=""
acc_data["pos"]=""

In [127]:
actual_data = []
x = 0
y = 34441
for i in range(0,5):
    actual_data.append(acc_data.iloc[x:y,:])
    x=y+1
    y=y+y

In [140]:
34441*4

137764

In [136]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [137]:
fin_data= []
file_name = "senti_chunck_"
for i in range(len(actual_data)):
    data = actual_data[i].copy()
    for j in range(len(data)):
        s=analyser.polarity_scores(data.iloc[j,2]) 
        data.iloc[j,5]=s['neg']
        data.iloc[j,6]=s['neu']
        data.iloc[j,7]=s['pos']
    f = file_name+str(i)+".csv"
    data.to_csv(f,sep=",",encoding='latin-1')
    fin_data.append(data)    
    

In [92]:
df1 = pd.DataFrame(fin_data[0])

In [90]:
fin_data[0].groupby('Senti').count()

,Date,Hour,Tweet content,Latitude,Longitude
Senti,,,,,
0,2763,2763,2763,2763,2763
4,31678,31678,31678,31678,31678


In [98]:
df1 = df1.append(fin_data[4])

In [100]:
df1.to_csv('full_us_senti.csv',sep=',',encoding='latin-1')

# Classifier Type 2

In [51]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.base import TransformerMixin

In [227]:
data1 = pd.read_csv(r'C:\Users\Tejal\Documents\Tejal\College\SSCI580\Project\Sentiment analysis\Train\Train.csv',encoding="latin-1")

In [228]:
train_data=data1.iloc[:,[1,3]].copy()

In [229]:
train_data = train_data[0:20000].copy()

In [230]:
test_data1=data1.iloc[:,[1,3]].copy()

In [231]:
test_data1 = test_data1.iloc[20001:20500,:].copy()

In [232]:
test_data1.shape

(499, 2)

In [233]:
test_data1.groupby('Sentiment').count()

,SentimentText
Sentiment,
0,234
1,265


In [193]:
unigram_bigram_clf = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="word",
                                   ngram_range=(1, 2),
                                   tokenizer=word_tokenize,
                                   # tokenizer=lambda text: mark_negation(word_tokenize(text)),
                                   preprocessor=lambda text: text.replace("<br />", " "),)),
    ('classifier', LinearSVC())
])

In [237]:
unigram_bigram_clf.fit(train_data.iloc[:,1], train_data.iloc[:,0])

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2),
        preprocessor=<function <lambda> a...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [238]:
unigram_bigram_clf.score(test_data1.iloc[:,1],test_data1.iloc[:,0])

0.74749498997995989

In [78]:
train_data1 = train_data1.drop(train_data1.index[2160])

In [105]:
x = unigram_bigram_clf.predict(train_data.iloc[200000:200009,1])

In [119]:
import numpy as np
x = np.array(x)
tw = np.array(train_data.iloc[200000:200009,1])

In [122]:
np_df = np.column_stack((tw,x))

In [128]:
df = pd.DataFrame(data=np_df,columns=['tweet','Senti'])

In [130]:
df.to_csv('senti_pred.csv',sep=',',encoding='latin-1')

In [120]:
test_data = pd.read_csv(r'C:\Users\Tejal\Documents\Tejal\College\SSCI580\Project\Sentiment analysis\tweet_15.csv',encoding='latin-1')

In [121]:
test_data['Tweet content']=test_data['Tweet content'].apply(lambda x: re.sub(r"http\S+", "", x))

In [122]:
test_data['Tweet content']=test_data['Tweet content'].apply(lambda x: re.sub(r"^@\S+", "", x))

In [123]:
test_data.axes

[RangeIndex(start=0, stop=3269, step=1),
 Index([u'Date', u'Hour', u'Tweet content', u'Latitude', u'Longitude'], dtype='object')]

In [245]:
senti = unigram_bigram_clf.predict(test_data.iloc[:,2])

In [246]:
len(senti)

3269

In [247]:
senti[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [248]:
df1 = pd.concat([test_data,pd.DataFrame(senti)],axis=1)

In [249]:
df1.to_csv('senti_pred_15_1.csv',sep=',',encoding='latin-1')

In [172]:
df1.iloc[:,5]

(3855, 6)

In [101]:
 vaderSentiment("I think spring finally has arrived!")

NameError: name 'vaderSentiment' is not defined

0.0